In [2]:
from pathlib import Path


folders = list(Path("../models/luminar_cnn/cross_validation_01JWBJNCZ7M8EMDX4C03280ZDR").iterdir())
folders

[PosixPath('../models/luminar_cnn/cross_validation_01JWBJNCZ7M8EMDX4C03280ZDR/student_essays'),
 PosixPath('../models/luminar_cnn/cross_validation_01JWBJNCZ7M8EMDX4C03280ZDR/euro_court_cases'),
 PosixPath('../models/luminar_cnn/cross_validation_01JWBJNCZ7M8EMDX4C03280ZDR/house_of_commons'),
 PosixPath('../models/luminar_cnn/cross_validation_01JWBJNCZ7M8EMDX4C03280ZDR/de'),
 PosixPath('../models/luminar_cnn/cross_validation_01JWBJNCZ7M8EMDX4C03280ZDR/spiegel_articles'),
 PosixPath('../models/luminar_cnn/cross_validation_01JWBJNCZ7M8EMDX4C03280ZDR/bundestag'),
 PosixPath('../models/luminar_cnn/cross_validation_01JWBJNCZ7M8EMDX4C03280ZDR/gutenberg_en'),
 PosixPath('../models/luminar_cnn/cross_validation_01JWBJNCZ7M8EMDX4C03280ZDR/arxiv_papers'),
 PosixPath('../models/luminar_cnn/cross_validation_01JWBJNCZ7M8EMDX4C03280ZDR/en'),
 PosixPath('../models/luminar_cnn/cross_validation_01JWBJNCZ7M8EMDX4C03280ZDR/blog_authorship_corpus'),
 PosixPath('../models/luminar_cnn/cross_validation_01JWBJNC

In [16]:
domain_map = {
    "blog_authorship_corpus": "Blog Authorship",
    "student_essays": "Student Essays",
    "cnn_news": "CNN News",
    "euro_court_cases": "Euro Court Cases",
    "house_of_commons": "House of Commons",
    "arxiv_papers": "ArXiv Papers",
    "gutenberg_en": "Gutenberg$_{en}$",
    "bundestag": "Bundestag$_{de}$",
    "spiegel_articles": "Spiegel$_{de}$",
    "de": "de",
    "en": "en",
}
order = {
    "Blog Authorship": 1,
    "Student Essays": 2,
    "CNN News": 3,
    "Euro Court Cases": 4,
    "House of Commons": 5,
    "ArXiv Papers": 6,
    "Gutenberg$_{en}$": 7,
    "Bundestag$_{de}$": 8,
    "Spiegel$_{de}$": 9,
    "de": 10,
    "en": 11,
}

In [ ]:
import json

import pandas as pd

results = []
for folder in folders:
    if not folder.is_dir():
        continue

    domain = folder.name.split("-", 1)[0]

    for cv_folder in folder.iterdir():
        metrics_test = cv_folder / "metrics_test.json"
        results.append({"domain": domain_map[domain], "order": order.get(domain_map[domain], 9999)} | json.load(metrics_test.open()))

results.sort(key=lambda x: order[x["domain"]])

results = pd.DataFrame(results).groupby("domain").agg(
    test_f1_weighted=("test_f1_weighted", "mean"),
    # test_accuracy=("test_accuracy", "mean"),
    test_roc_auc=("test_roc_auc", "mean"),
    order=("order", "first"),
    # test_f1_ai=("test_f1_ai", "mean"),
    # test_f1_human=("test_f1_human", "mean"),
    # test_f1_weighted_median=("test_f1_weighted_median", "mean"),
).round(3).sort_values("order")
results


,test_f1_weighted,test_roc_auc,order
domain,,,
Blog Authorship,0.791,0.884,1
Student Essays,0.996,1.000,2
CNN News,0.938,0.984,3
Euro Court Cases,0.977,0.997,4
House of Commons,0.947,0.989,5
ArXiv Papers,0.980,0.996,6
Gutenberg$_{en}$,0.905,0.964,7
Bundestag$_{de}$,0.963,0.992,8
Spiegel$_{de}$,0.841,0.921,9


In [34]:
print(results.to_latex(float_format="%.3f", escape=False))

\begin{tabular}{lrrr}
\toprule
 & test_f1_weighted & test_roc_auc & order \\
domain &  &  &  \\
\midrule
Blog Authorship & 0.791 & 0.884 & 1 \\
Student Essays & 0.996 & 1.000 & 2 \\
CNN News & 0.938 & 0.984 & 3 \\
Euro Court Cases & 0.977 & 0.997 & 4 \\
House of Commons & 0.947 & 0.989 & 5 \\
ArXiv Papers & 0.980 & 0.996 & 6 \\
Gutenberg$_{en}$ & 0.905 & 0.964 & 7 \\
Bundestag$_{de}$ & 0.963 & 0.992 & 8 \\
Spiegel$_{de}$ & 0.841 & 0.921 & 9 \\
de & 0.883 & 0.958 & 10 \\
en & 0.959 & 0.993 & 11 \\
\bottomrule
\end{tabular}



In [5]:
import json

import pandas as pd


results = []
for folder in folders:
    if not folder.is_dir():
        continue

    domain = folder.name.split("-", 1)[0]

    for cv_folder in folder.iterdir():
        metrics_unmatched = cv_folder / "metrics_unmatched.json"
        results.append({"domain": domain} | json.load(metrics_unmatched.open()))

results = pd.DataFrame(results)
print(results.columns)
results.groupby("domain").agg(
    unmatched_f1_ai=("unmatched_f1_ai", "mean"),
    unmatched_f1_human=("unmatched_f1_human", "mean"),
    unmatched_f1_weighted=("unmatched_f1_weighted", "mean"),
    unmatched_accuracy=("unmatched_accuracy", "mean"),
    unmatched_roc_auc=("unmatched_roc_auc", "mean"),
    # unmatched_f1_weighted_mean=("unmatched_f1_weighted_mean", "mean"),
).mul(100).round(2)

Index(['domain', 'unmatched_loss', 'unmatched_f1_weighted',
       'unmatched_accuracy', 'unmatched_roc_auc', 'unmatched_f1_human',
       'unmatched_f1_ai', 'unmatched_n_samples', 'unmatched_f1_weighted_mean',
       'unmatched_accuracy_mean', 'unmatched_roc_auc_mean',
       'unmatched_f1_human_mean', 'unmatched_f1_ai_mean',
       'unmatched_threshold_mean', 'unmatched_n_samples_human',
       'unmatched_n_samples_ai', 'unmatched_runtime',
       'unmatched_samples_per_second', 'unmatched_steps_per_second', 'epoch'],
      dtype='object')


,unmatched_f1_ai,unmatched_f1_human,unmatched_f1_weighted,unmatched_accuracy,unmatched_roc_auc
domain,,,,,
arxiv_papers,87.13,89.72,88.45,88.57,97.79
blog_authorship_corpus,53.89,73.14,65.02,66.09,71.45
bundestag,62.58,78.78,70.86,72.94,90.20
cnn_news,76.25,84.19,80.54,81.03,92.93
de,66.76,79.09,73.26,74.35,87.14
en,69.34,80.05,75.08,75.84,87.79
euro_court_cases,90.60,91.92,91.26,91.31,98.62
gutenberg_en,71.09,79.63,75.39,76.13,87.77
house_of_commons,69.63,80.11,74.99,75.98,89.25
